# Data

Prepares the raw International Classification of Diseases (ICD) data into tabular `.csv` files.

Optionally, uploads the prepared data into a database.

## Libraries

In [1]:
import numpy as np
import os
import pandas as pd
import subprocess
import yaml

from edotenv import load_edotenv
from pandas.api.types import is_numeric_dtype
from pathlib import Path
from sqlalchemy import create_engine, text, inspect

## Config

Load settings from the `config.yml` file.

In [2]:
with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)
    
version = config['Version']

## Save Login (Optional)

To connect to the database, you need to ensure that you have saved your login with successfully with `bin/login.bat` or `bin/login.sh`:

In Windows, run:

```
bin\login
```

In Linux/Mac OS, run:

```
source bin/login.sh
```

If these commands run successfully, a `.env` file will be created.

## Database Connection (Optional)

Connect to the database if provided.

In [3]:
if 'ICD_UPLOAD_DB_URL' in os.environ:
    load_edotenv('../.env')
    uengine = create_engine(os.environ['ICD_UPLOAD_DB_URL'])

**Note**: If this fails, try `bin/login.bat` or `bin/login.sh` again.

## Process Data

Process the datasets by:

1. Reading the raw data from a file
2. Removing columns that are not needed
3. Renaming original columns to desired column names
4. Cleaning values by:
    a. Removing leading dashes from values
    b. Converting data types

In [4]:
# Create folder to store each dataset
Path('../data').mkdir(exist_ok=True)

# Process each dataset
datasets = {}
for data in config['Data']:
    
    # Get data info
    name = data['Name']
    file = Path(data['File'])
    print(f'Processing {name}...')
    
    # 1. Read raw data
    ext = file.suffix.lower()
    if ext in ['.xls', '.xlsx']:
        df = pd.read_excel(file)
    elif ext == '.csv':
        df = pd.read_csv(file)
    else:
        raise ValueError(f'Extension {ext} not supported.')
    
    # 2-4. Remove unneeded cols and rename
    if 'Columns' in data:
        
        # 2. Keep cols
        print(f'Extracting columns {name}...')
        keep_cols = [c['Name'] for c in data['Columns']]
        df = df[keep_cols]
        
        # 3. Rename cols
        print(f'Renaming columns {name}...')
        rename_cols = {c['Name']: c.get('Rename', c['Name']) for c in data['Columns']}
        df = df.rename(columns=rename_cols)
        
        # 4. Preprocess cols
        for c in data['Columns']:
            cname = c.get('Rename', c['Name'])
            
            # 4a. Remove leading dashes
            if c.get('Remove Leading Dashes', False):
                print(f'Removing leading dashes ({cname})...')
                df[cname] = df[cname].str.lstrip(' -')
                
            # 4b. Convert data type
            if 'Type' in c:
                print(f'Converting data type ({cname})...')
                df[cname] = df[cname].astype(c['Type'])
    
    # Set processed dataset
    datasets[name] = df
    print(f'Processed {name}!')

Processing icd10...
Extracting columns icd10...
Renaming columns icd10...
Processed icd10!
Processing icd11...
Extracting columns icd11...
Renaming columns icd11...
Removing leading dashes (title)...
Processed icd11!


Preview a dataset.

In [5]:
preview = list(datasets.keys())[0]
datasets[preview]

,kind,kind_depth,chapter,code,title
0,chapter,1,I,I,Certain infectious and parasitic diseases
1,block,1,I,A00-A09,Intestinal infectious diseases
2,block,1,I,A00-A09,Intestinal infectious diseases
3,block,1,I,A00-A09,Intestinal infectious diseases
4,block,1,I,A00-A09,Intestinal infectious diseases
...,...,...,...,...,...
15555,category,2,XXII,U84.9,Resistance to unspecified antimicrobial drugs
15556,category,2,XXII,U84.9,Resistance to unspecified antimicrobial drugs
15557,category,2,XXII,U84.9,Resistance to unspecified antimicrobial drugs
15558,category,2,XXII,U84.9,Resistance to unspecified antimicrobial drugs


## Data Descriptions and Dictionaries

1. Create data descriptions from `config.yml` using the `Description` key for each item under the `Data` key.
2. Also create data dictionaries from `config.yml` using the `Columns` key for each item under the `Data` key.

In [6]:
# Create folder to store ddict
Path('../data').mkdir(exist_ok=True)

# Create descriptions and ddict of each dataset
ddict = []
ddescribe = []
for data in config['Data']:
    
    # Get processed dataset
    name = data['Name']
    df = datasets[name]
    
    # 1. Add dataset description to ddescribe
    ddescribe.append({
        'dataset': name,
        'columns': len(df.columns),
        'rows': len(df),
        'description': data['Description']
    })
    
    # 2a. Calc col stats for ddict
    dd = df.describe(include='all', datetime_is_numeric=True).transpose()
    dd = dd.reset_index()
    dd = dd.rename(columns={'index': 'column'})
    dd.insert(0, 'dataset', name)
    
    # 2b. Add col descriptions to ddict
    dd.insert(2, 'type', dd.dtypes)
    dd.insert(3, 'character_length', pd.Series(dtype='int'))
    dd.insert(4, 'description', pd.Series(dtype='str'))
    for c in data['Columns']:
        cname = c.get('Rename', c['Name'])
        dd.loc[dd['column'] == cname, ['type']] = c.get('Type', str(df[cname].dtype))
        dd.loc[dd['column'] == cname, ['description']] = c.get('Description', None)
        dd.loc[dd['column'] == cname, ['character_length']] = c.get('Character Length', None)
    ddict.append(dd)
    
# Gather descriptions and ddict
ddescribe = pd.DataFrame(ddescribe)
ddict = pd.concat(ddict)

Preview data descriptions.

In [7]:
ddescribe

,dataset,columns,rows,description
0,icd10,5,15560,International Classification of Diseases Revis...
1,icd11,17,35459,International Classification of Diseases Revis...


Preview data dictionary.

In [8]:
ddict

,dataset,column,type,description,count,unique,top,freq,mean,std,min,25%,50%,75%,max
0,icd10,kind,object,"ICD-10 entity kind. One of chapter, block, or ...",15560,4,category,13793,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,icd10,kind_depth,int64,"ICD-10 depth for kind. For example, a category...",15560.0,NaN,NaN,NaN,1.79428,0.413047,1.0,2.0,2.0,2.0,3.0
2,icd10,chapter,object,ICD-10 chapter for the entity.,15560,22,XX,1692,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,icd10,code,object,ICD-10 code for the entity.,15560,12597,XXI,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,icd10,title,object,ICD-10 title of the entity.,15560,12536,Factors influencing health status and contact ...,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,icd11,kind,object,"ICD-11 entity kind. One of chapter, block, or ...",35459,3,category,34079,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,icd11,kind_depth,int64,"ICD-11 depth for kind. For example, a category...",35459.0,NaN,NaN,NaN,2.050312,1.045853,1.0,1.0,2.0,3.0,8.0
2,icd11,chapter,object,ICD-11 chapter for the entity.,35459,28,X,16964,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,icd11,code,object,ICD-11 code for the entity. Note that the grou...,34079,34079,1A00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,icd11,title,object,ICD-11 title of the entity.,35459,35401,Acute myeloid leukaemia without maturation,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Datasets Upload (Optional)

Upload datasets to created tables in the PostgreSQL database.

In [ ]:
if 'ICD_UPLOAD_DB_URL' in os.environ:
    
    for data in config['Data']:
    
        # Get info from config for dataset
        version = data['Version']
        name = data['Name']
        table = f'{name}_v{version}'
        schema = data['Schema'] if 'Schema' in data else None

        # Upload to db
        print(f'Uploading {table}...')
        if not inspect(uengine).has_table(table):
            
            # Upload to db depending on whether it has geodata
            df = datasets[name]
            if 'Geometry Column' in data:
                df.to_postgis(table, uengine, schema=schema, index=False)
            else:
                df.to_sql(table, uengine, schema=schema, index=False)
            print(f'Uploaded {table}!')
            
        else:
            
            # Skip upload if table exists
            print(f'Table {table} exists - skipping!')
            

## Comments Upload (Optional)

Add table and column comments to uploaded dataset tables by:

1. Generating SQL for dataset table comment
2. Generating SQL for dataset column comments
3. Executing generated SQL statements above

In [ ]:
if 'ICD_UPLOAD_DB_URL' in os.environ:
    
    sql = {}
    for data in config['Data']:

        # Get info from config for dataset
        version = data['Version']
        name = data['Name']
        table = f'{name}_v{version}'

        # Add schema if avail
        if 'Schema' in data:
            schema = data['Schema']
            table = f'{schema}.{table}'

        # Get ddict for dataset
        dd = ddict[ddict['dataset'] == name]

        # 1. Create sql for table comment
        squote = "'"
        dbquotes = "''"
        description = ddescribe[ddescribe['dataset'] == name]['description']
        description = description.tolist()[0].replace(squote, dbquotes)
        comment_query = f"COMMENT ON TABLE {table} IS '{description}';"

        # 2. Create sql for column comments
        ncols = dd.shape[0]
        col_query = [f"COMMENT ON COLUMN {table}.{r['column']} IS '{str(r['description']).replace(squote, dbquotes)}';" for i, r in dd.iterrows()]
        col_query = '\n'.join(col_query)

        # 3a. Add table and col comment statements
        query = f'--- {table} table comment\n' + comment_query \
            + f'\n\n--- {table} column comments (n={ncols})\n' + col_query
        sql[name] = query

    # 3b. Combine and execute comment statements
    comment_sql = '\n\n'.join(q for dataset, q in sql.items())
    with uengine.connect() as connection:
        connection.execute(text(comment_sql))
        connection.execute('COMMIT;')

Save comments sql to file `mds_comments.sql` in `downloads` folder.

In [ ]:
if 'ICD_UPLOAD_DB_URL' in os.environ:
    
    # Create folder to store database outputs
    Path('../downloads/database').mkdir(exist_ok=True)
    
    # Save comments sql for tables
    with open(f'../downloads/database/mds_comments_v{version}.sql', 'w') as file:
        file.write(comment_sql)

## View Uploads (Optional)

Create views for the uploaded tables with accompanying view/column comments.

In [ ]:
if 'ICD_UPLOAD_DB_URL' in os.environ:
    
    sql = {}
    for data in config['Data']:

        # Get info from config for dataset
        version = data['Version']
        name = data['Name']
        table = f'{name}_v{version}'

        # Add schema if avail
        if 'Schema' in data:
            schema = data['Schema']
            table = f'{schema}.{table}'
            
        # Get ddict for dataset
        dd = ddict[ddict['dataset'] == name]

        # 1. Create view query
        view_query = f'CREATE OR REPLACE VIEW {name} AS (SELECT * FROM {table});'
        
        # 2. Create sql for view comment
        squote = "'"
        dbquotes = "''"
        description = ddescribe[ddescribe['dataset'] == name]['description']
        description = description.tolist()[0].replace(squote, dbquotes)
        comment_query = f"COMMENT ON VIEW {name} IS '{description}';"

        # 3. Create sql for column comments
        ncols = dd.shape[0]
        col_query = [f"COMMENT ON COLUMN {name}.{r['column']} IS '{str(r['description']).replace(squote, dbquotes)}';" for i, r in dd.iterrows()]
        col_query = '\n'.join(col_query)
        
        # 3a. Add view, comment, and column statements
        sql[name] = f'--- {name} view\n\n' + view_query \
            + f'\n\n--- {name} view comment\n\n' \
            + comment_query \
            + f'\n\n--- {name} view column comments (n={ncols})\n\n' \
            + col_query

    # 3b. Combine and execute comment statements
    view_sql = '\n\n'.join([q for dataset, q in sql.items()])
    with uengine.connect() as connection:
        connection.execute(text(view_sql))
        connection.execute('COMMIT;')

Save views sql to file `mds_views.sql` in `downloads` folder.

In [ ]:
if 'ICD_UPLOAD_DB_URL' in os.environ:
    with open(f'../downloads/database/mds_views_v{version}.sql', 'w') as file:
        file.write(view_sql)

## Dump Uploaded Tables (Optional)

Create database dumps of the uploaded tables and views as backup.

In [ ]:
if 'ICD_UPLOAD_DB_URL' in os.environ:
    
    # Setup env for running pgdumps
    os.environ['PGHOST'] = uengine.url.host
    os.environ['PGPORT'] = str(uengine.url.port)
    os.environ['PGDATABASE'] = uengine.url.database
    os.environ['PGUSER'] = uengine.url.username
    os.environ['PGPASSWORD'] = uengine.url.password
            
    # Dump table to downloads
    print('Dumping uploaded tables...')
    targs = ' '.join(['-t ' + data['Name'] + '_v' + str(data['Version']) for data in config['Data']])
    targs = targs.split()
    cmd = ['pg_dump', '--no-owner', '-F', 'c'] + targs + ['-f', f'mds_tables_v{version}.dump']
    subprocess.call(cmd, env=os.environ, cwd='../downloads/database')
    print(f'Dumping uploaded tables complete!')

Close database connections.

In [ ]:
if 'ICD_UPLOAD_DB_URL' in os.environ:
    uengine.dispose()